In [6]:
# import modules and define functions
from pyscf import gto, scf, dft
from pyscf.geomopt.geometric_solver import optimize
import pandas as pd
import numpy as np
import os
from functools import reduce
os.environ['OMP_NUM_THREADS'] = "1" 

# Koopman's theorem

In [7]:
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74'
basis = 'def2-qzvpp'
h2_0_1 = gto.Mole()
h2_0_1.build(atom=atom,basis=basis,charge=0,spin=0)

In [8]:
# SCF calculations
# HF
h2_0_1_uhf = scf.UHF(h2_0_1).run()

converged SCF energy = -1.13356268665593  <S^2> = 1.5787371e-13  2S+1 = 1


In [9]:
# Koopman's IP for H2
mo_occ = h2_0_1_uhf.mo_occ
mo_energy = h2_0_1_uhf.mo_energy
e_homo_h2 = mo_energy[mo_occ>0][-1]
print('HOMO energy of H2 at HF level is %.4f Eh' %(e_homo_h2))

HOMO energy of H2 at HF level is -0.5949 Eh


In [10]:
# calculate IP for H2
h2_0_1_uhf = scf.UHF(gto.Mole().build(atom=atom,basis=basis,charge=0,spin=0)).run()
h2_1_2_uhf = scf.UHF(gto.Mole().build(atom=atom,basis=basis,charge=1,spin=1)).run()
s2_h2 = h2_1_2_uhf.spin_square()[0]
IP_H2 = (h2_0_1_uhf.e_tot - h2_1_2_uhf.e_tot)
print('Ionization potential calculated from e_diff between H2 and H2+ is %.4f Eh' %(IP_H2))

converged SCF energy = -1.13356268665593  <S^2> = 1.5787371e-13  2S+1 = 1
converged SCF energy = -0.569681447884232  <S^2> = 0.75  2S+1 = 2
Ionization potential calculated from e_diff between H2 and H2+ is -0.5639 Eh


In [11]:
# calculate MO and IP for CO and N2
# CO
co_0_1 = gto.Mole().build(atom='C 0 0 0; O 0 0 1.13',basis=basis,charge=0,spin=0)
co_1_2 = gto.Mole().build(atom='C 0 0 0; O 0 0 1.13',basis=basis,charge=1,spin=1)
co_0_1_uhf = scf.UHF(co_0_1).run()
co_1_2_uhf = scf.UHF(co_1_2).run()
s2_co = co_1_2_uhf.spin_square()[0]
mo_occ = co_0_1_uhf.mo_occ
mo_energy = co_0_1_uhf.mo_energy
e_homo_co = mo_energy[mo_occ>0][-1]
IP_CO = (co_0_1_uhf.e_tot - co_1_2_uhf.e_tot)
# N2
# calculate MO and IP for CO and N2
n2_0_1 = gto.Mole().build(atom='N 0 0 0; N 0 0 1.098',basis=basis,charge=0,spin=0)
n2_1_2 = gto.Mole().build(atom='N 0 0 0; N 0 0 1.098',basis=basis,charge=1,spin=1)
n2_0_1_uhf = scf.UHF(n2_0_1).run()
n2_1_2_uhf = scf.UHF(n2_1_2).run()
s2_n2 = n2_1_2_uhf.spin_square()[0]
mo_occ = n2_0_1_uhf.mo_occ
mo_energy = n2_0_1_uhf.mo_energy
e_homo_n2 = mo_energy[mo_occ>0][-1]
IP_N2 = n2_0_1_uhf.e_tot - n2_1_2_uhf.e_tot

converged SCF energy = -112.790032352633  <S^2> = 4.2877701e-11  2S+1 = 1
converged SCF energy = -112.309872795567  <S^2> = 1.0136161  2S+1 = 2.2482136
converged SCF energy = -108.992256057247  <S^2> = 1.018563e-11  2S+1 = 1
converged SCF energy = -108.418008939501  <S^2> = 0.76732342  2S+1 = 2.017249


In [26]:
data = [['H\u2082', '0.584', e_homo_h2, IP_H2, s2_h2],
        ['CO','0.510', e_homo_co, IP_CO, s2_co],
        ['N\u2082','0.573', e_homo_n2, IP_N2, s2_n2]]
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(pd.DataFrame(data, columns=['Molecules', 'Expt', 'Koopman', 'IP', '<S^2>']))

  Molecules   Expt  Koopman     IP  <S^2>
0        H₂  0.584   -0.595 -0.564  0.750
1        CO  0.510   -0.555 -0.480  1.014
2        N₂  0.573   -0.613 -0.574  0.767


# Brillouin theorem

In [13]:
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74'
basis = '6-31g'
h2_0_1 = gto.Mole()
h2_0_1.build(atom=atom,basis=basis,charge=0,spin=0)
h2_0_1_rhf = scf.RHF(h2_0_1).run()

converged SCF energy = -1.12675531719693


In [14]:
# get the mo_coeffs and the 2e and fock matrix elements
mo_coeff = h2_0_1_rhf.mo_coeff
mo_occ = h2_0_1_rhf.mo_occ
h = h2_0_1_rhf.get_hcore()
veff = h2_0_1_rhf.get_veff()
f = h2_0_1_rhf.get_fock()

In [15]:
# get H and G
F = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, f, mo_coeff])

In [16]:
colnames = ['\u03c8' + '%s' %(i) for i in range(F.shape[0])]
pd.set_option('display.chop_threshold', 0.000001)
pd.set_option('display.float_format', lambda x: '%.6f' % x)
df = pd.DataFrame(F, columns=colnames, index=colnames)
print(df)


          ψ0        ψ1        ψ2       ψ3
ψ0 -0.595817 -0.000000 -0.000000 0.000000
ψ1 -0.000000  0.238473 -0.000000 0.000000
ψ2 -0.000000 -0.000000  0.774723 0.000000
ψ3  0.000000  0.000000  0.000000 1.404412
